# Image Morphing 
The goal of this exercise is to present a simple method for image morphing. We will work with pairs of images containing faces which are annotated with facial landmarks. The goal is to morph (transform) a face into another other face by creating a smooth transition between the source image and the destination image. The transition contains fake images obtained using geometric and photometric interpolation based on regions (triangles) selected from the two images. The method can be easily extended to other class of objects (different than faces).

<img src="images\final_result.gif" width=100 />

In achieving our goal, we will follow the next steps:
1. compute the Delauney triangulation based on the facial landmarks in the source image, partinioning the face in triangles. We transfer this triangulation to the destination image for corresponding facial landmarks;
2. compute the affine transformation between pairs of corresponding triangles from the two images;
3. create fake triangles based on geometric and photometric interpolation of triangles selected  from the source and destination images

<table width="950px">
<tr>
<th><center>Source - real</center></th>
<th><center>fake</center></th> 
<th><center>fake</center></th> 
<th><center>fake</center></th> 
<th><center>fake</center></th> 
<th><center>fake</center></th> 
<th><center>Destination - real</center></th> 
</tr>
<tr>
<td><img src="images\img_0.jpg" width=100 /></td>
<td><img src="images\img_10.jpg" width=100 /></td>
<td><img src="images\img_20.jpg" width=100 /></td>
<td><img src="images\img_30.jpg" width=100 /></td>
<td><img src="images\img_40.jpg" width=100 /></td>
<td><img src="images\img_50.jpg" width=100 /></td>
<td><img src="images\img_60.jpg" width=100 /></td> 
</tr>
</table>   

In [2]:
import numpy as np
import cv2 as cv
from scipy.spatial import Delaunay
import scipy.io

In [3]:
def show_image_points(img,points):    
    img_ = img.copy()
    for p in points:
        cv.circle(img_, (p[0], p[1]), 3, (0, 255, 0), -1)    
    cv.imshow("image",img_)
    cv.waitKey(0) # waits until a key is pressed
    cv.destroyAllWindows() # destroys the window showing image
    return img_

In [4]:
#show the annotated facial landmarks on each image
img1 = cv.imread('images/bogdan_crop.jpg'); #source image
temp = scipy.io.loadmat('data/bogdan_points.mat')
p1 = temp['points']
p1 = np.uint64(p1)
img1_ = show_image_points(img1, p1)

img2 = cv.imread('images/clooney_crop.jpg');#destination image
temp = scipy.io.loadmat('data/clooney_points.mat')
p2 = temp['points']
p2 = np.uint64(p2)
img2_ = show_image_points(img2, p2)

In [5]:
cv.imshow("img1_",img1_)
cv.imshow("img2_",img2_)
cv.waitKey(0)
cv.destroyAllWindows()

In [6]:
#compute the Delauney triangulation based on the facial landmarks in the source image
#the face is partitioned in triangles
triangulation = Delaunay(p1)
print("We have obtained ", str(triangulation.simplices.shape[0]), " triangles")
#print(triangulation.simplices) #show for each triangle the index of each vertex

We have obtained  70  triangles


In [7]:
#plot the triangles partitioning the face, each triangle with a different color
def show_image_triangulated(img,points,triangulation):
    img_t = img.copy()
    colors = [[255,0,0],[0,255,0],[0,0,255],[255,255,0],[255,0,255],[0,255,255],[0,0,0],[128,255,255],[255,128,255],[255,255,128]]
    number_colors = len(colors)    
    
    for idx,t in enumerate(triangulation.simplices):
        point1 = points[t[0]]
        #cv.circle(img, (point1[0], point1[1]), 3, (0, 255, 0), -1)
        point2 = points[t[1]]
        #cv.circle(img, (point2[0], point2[1]), 3, (0, 255, 0), -1)
        point3 = points[t[2]]
        #cv.circle(img, (point3[0], point3[1]), 3, (0, 255, 0), -1)
        
        cv.line(img_t, (point1[0],point1[1]), (point2[0],point2[1]), colors[idx % number_colors], 3)
        cv.line(img_t, (point2[0],point2[1]), (point3[0],point3[1]), colors[idx % number_colors], 3)
        cv.line(img_t, (point3[0],point3[1]), (point1[0],point1[1]), colors[idx % number_colors], 3)            
        
    cv.imshow("image with triangles",img_t)
    cv.waitKey(0) # waits until a key is pressed
    cv.destroyAllWindows() # destroys the window showing image
    return img_t

img_t_1 = show_image_triangulated(img1,p1,triangulation)
#transfer the triangulation to the destination image (we migh have triangles which overlap)
img_t_2 = show_image_triangulated(img2,p2,triangulation)

In [8]:
cv.imshow("image1",img_t_1)
cv.imshow("image2",img_t_2)
cv.waitKey(0)
cv.destroyAllWindows()

Compute the affine transformation T between two triangles ABC and DEF using the formula derived in lecture 9. We use the proxy triangle with vertices (0,1), (0,0) and (1,0) to get affine transformation T1 and T2. Then, we have that T = T2 * inv(T1)
<table width="950px">
<tr>
<td><img src="images\affine_transformation_1.png" width=450 /></td>
<td><img src="images\affine_transformation_2.png" width=450 /></td>    
</tr>
</table>   

In [11]:
def compute_affine_transformation(triangle_1, triangle_2):# 
# find the affine transformation matrix T that transforms vertices of triangle_1 in vertices of triangle_2
# follow the lecture: compute first T1, T2, than T = T2 * inv(T1) 
    
    ...
    
    T = T2 @ np.linalg.inv(T1)
        
    return T

In [ ]:
#sanity check for the first triangle with vertices = points 18, 17, 19

#take triangle ABC: A = p1[18], B = p1[17], C = p1[19]
triangle_1 = np.float32([p1[18],p1[17],p1[19]])

#take triangle DEF: D= p2[18], E = p2[17], F = p2[19]
triangle_2 = np.float32([p2[18],p2[17],p2[19]])

print("Triangle ABC: ", triangle_1,"\n")
print("Triangle DEF:", triangle_2,"\n")

T = compute_affine_transformation(triangle_1,triangle_2)
print("The affine transformation matrix T = ", T,"\n")

#verify that T transforms A in D
print("Verifying that T trasforms A in D: A = ",p1[18], " D = ", p2[18])
print("Apply T to point A. We obtain ",T.dot([p1[18][0],p1[18][1],1]), "\n")

#verify that T transforms B in E
print("Verifying that T transforms B in E: B = ",p1[17], " E = ", p2[17])
print("Apply T to point B. We obtain ",T.dot([p1[17][0],p1[17][1],1]),"\n")

#verify that T transforms C in F
print("Verifying that T transforms C in F: C = ",p1[19], " F = ", p2[19])
print("Apply T to point C. We obtain ",T.dot([p1[19][0],p1[19][1],1]),"\n")

#verify that T transforms middle point of AB in middle point of DE
mAB = 0.5*p1[17] + 0.5*p1[18]  
mDE = 0.5*p2[17] + 0.5*p2[18]
print("middle point of AB = ", mAB)
print("middle points of DE = ", mDE)
print("Verifying that T transforms middle point of AB in middle point of DE: mAB = ",mAB, " mDE = ", mDE)
print("Apply T to point mAB. We obtain ",T.dot([mAB[0],mAB[1],1]),"\n")

In [9]:
# Warps and alpha blends triangular regions from img1 and img2 to img
def morph_triangles(img1, img2, img_morphed, t1, t2, alpha,show_images) :

    #compute the coordinates of the triangle in the morphed image as average weighting
    t = (1-alpha) * t1 + alpha * t2
    
    # Find bounding rectangle for each triangle
    r1 = cv.boundingRect(np.float32([t1]))
    r2 = cv.boundingRect(np.float32([t2]))
    r = cv.boundingRect(np.float32([t]))

    # Offset points by left top corner of the respective rectangles
    t1_rect = []
    t2_rect = []
    t_rect = []

    for i in range(0, 3):
        t_rect.append(((t[i][0] - r[0]),(t[i][1] - r[1])))
        t1_rect.append(((t1[i][0] - r1[0]),(t1[i][1] - r1[1])))
        t2_rect.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))


    # Get mask by filling triangle
    mask = np.zeros((r[3], r[2], 3), dtype = np.float32)
    cv.fillConvexPoly(mask, np.int32(t_rect), (1.0, 1.0, 1.0), 16, 0);

    # Apply warpImage to small rectangular patches
    img1_rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
    img2_rect = img2[r2[1]:r2[1] + r2[3], r2[0]:r2[0] + r2[2]]

    size = (r[2], r[3])
    
    T = compute_affine_transformation(t1_rect, t_rect)        
    warp_img1 = cv.warpAffine(img1_rect, T[:2], (size[0], size[1]), None, flags=cv.INTER_LINEAR, borderMode=cv.BORDER_REFLECT_101 )
    
    T = compute_affine_transformation(t2_rect, t_rect)    
    warp_img2 = cv.warpAffine(img2_rect, T[:2], (size[0], size[1]), None, flags=cv.INTER_LINEAR, borderMode=cv.BORDER_REFLECT_101 )
       

    # Alpha blend rectangular patches
    img_rect = np.uint8((1.0 - alpha) * warp_img1 + alpha * warp_img2)
    
    # Copy triangular region of the rectangular patch to the output image
    img_morphed[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] = img_morphed[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] * ( 1 - mask ) + img_rect * mask
    
    if show_images:
        cv.imshow("img1_Rect",img1_rect)
        cv.imshow("warped image 1",warp_img1)
        cv.imshow("img2_Rect",img2_rect)
        cv.imshow("warped image 2",warp_img2)
        cv.imshow("blend rectangular patches",img_rect)
        cv.imshow("mask",mask)
        cv.imshow("img_morphed",img_morphed) 
        cv.waitKey(0)
        cv.destroyAllWindows()
                
    return img_morphed

In [ ]:
#do a demo for just one triangle
#morph a triangle to the corresponding triangle
#take the pairs of triangles from line 20, with vertices the landmarks: 31 33  7

#take triangle ABC: A = p1[31], B = p1[33], C = p1[7]
t1 = np.float32([p1[31],p1[33],p1[7]])

#take triangle DEF: D= p2[31], E = p2[33], F = p2[7]
t2 = np.float32([p2[31],p2[33],p2[7]])

alpha = 0.5
img_morphed = np.zeros(img1.shape, dtype = img1.dtype)
img_morphed = morph_triangles(img1, img2, img_morphed, t1, t2, alpha,1)

In [28]:
def morph_image(img1,img2,p1,p2,triangulation,alpha,show_images):
    
    img_morph = np.zeros(img1.shape, dtype = img1.dtype)
    
    ...
        
    return img_morph

In [29]:
img1 = cv.imread('images/bogdan_crop.jpg'); #source image
img2 = cv.imread('images/clooney_crop.jpg');#destination image

temp = scipy.io.loadmat('data/bogdan_points.mat')
p1 = temp['points']
p1 = np.uint64(p1)

temp = scipy.io.loadmat('data/clooney_points.mat')
p2 = temp['points']
p2 = np.uint64(p2)

triangulation = Delaunay(p1)
number_steps = 50

for step in range(0,number_steps+1):
    alpha = step/number_steps
    
    #print("alpha = ", alpha)
    img_morph = morph_image(img1, img2, p1, p2, triangulation, alpha,0)
    
    # Display Result
    cv.imwrite("gif/image_" + str(step) +  ".png",img_morph)
    cv.imshow("Morphed Face", np.uint8(img_morph))
    cv.waitKey(500)
    cv.destroyAllWindows()

In [ ]:
#go to https://gifmaker.me/ for making a gif
#could also install some libraries to make gif in opencv